In [1]:
using Query
using DataFrames
using CSVFiles

In [53]:

dist_df = DataFrame(load("results/raw/dist_full.csv" ))



for filename in ["ord.csv", "full.csv", "extrapo.csv"]
    df = DataFrame(load(joinpath("results", "raw", "dist_"*filename)))

    res_form = @from ii in df begin
        @select {method="Distribution Mean " .* ii.method, mse=ii.mse_to_distmean}
    end

    save(joinpath("results", "distpoint_"*filename), res_form)
end
###############################


In [68]:
Base.String() = ""
Base.Float64() = NaN



method_name(method)::String = join(
    filter(x->x ∉ ["Extrapolating", "Non-extrapolating"],
           split(method)), " ")





extrapo_res(df) = df |> @filter(contains(_.method, "Extrapolating"))
nonextrapo_res(df) = df |> @filter(contains(_.method, "Non-extrapolating"))

###########################
filename =  "results/raw/dist_extrapo.csv" 
df = DataFrame(load(filename))

res_form = @from nonext in nonextrapo_res(df) begin
    @let base_method = method_name(nonext.method)
    @left_outer_join ext in extrapo_res(df) on base_method equals method_name(ext.method)
    @select {method=base_method, nonextrapolating_perp=nonext.perp, extrapolating_perp = ext.perp}
end

save("results/res_dist_extrapo.csv", res_form)
###############################

###########################

point_extrapos = map(["raw/point_extrapo.csv", "distpoint_extrapo.csv"]) do filename
    df = DataFrame(load(joinpath("results", filename)))

    res_form = @from nonext in nonextrapo_res(df) begin
        @let base_method = method_name(nonext.method)
        @left_outer_join ext in extrapo_res(df) on base_method equals method_name(ext.method)
        @select {method=base_method,
            nonextrapolating_mse=nonext.mse,
            extrapolating_mse= ext.mse}
        @collect DataFrame
    end
end
###############################

save("results/res_point_comb_extrapo.csv", vcat(point_extrapos...))

In [70]:
save("results/res_point_comb_full.csv",
    vcat(DataFrame.(load.(["results/raw/point_full.csv", "results/distpoint_full.csv"]))...)
    )
save("results/res_point_comb_ord.csv",
    vcat(DataFrame.(load.(["results/raw/point_ord.csv", "results/distpoint_ord.csv"]))...)
    )


In [72]:
cp("results/raw/dist_full.csv","results/res_dist_full.csv", remove_destination=true)
cp("results/raw/dist_ord.csv","results/res_dist_ord.csv", remove_destination=true)